In [10]:
!pip install pylhe

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
import pylhe
import math

In [12]:
# Pi0 mass
Mpi0 = 0.1349768

# epsilon^2
eps2 = [1e-10, 1.4e-10, 2e-10, 1e-9, 1-3e-9, 1e-8]

# Dark photon mass
mA = [1e-1, 5e-2, 1e-2]

def scale_factor(eps2,mA):
  return ( 2 * eps2 * (1 - mA**2/Mpi0**2)**3 )

# factor to scale signal to a year
def signal_factor(eps2, mA, signal):
  return scale_factor(eps2, mA) * 2e22 / len(signal)

# factor to adjust signal to an eps2
def eps_factor(eps2):
  return eps2 / (5e-3)**2

# factor to scale background to a year
bckg_factor = 1.1e21 / 515500

# Turning all madDump data into dataframes

In [13]:
eventsM1 = []
eventsM2 = []
eventsM3 = []

with open("events_mass1.lhe", "r") as fM1:
    linesM1 = fM1.readlines()

with open("events_mass2.lhe", "r") as fM2:
   linesM2 = fM2.readlines()

with open("events_mass3.lhe", "r") as fM3:
    linesM3 = fM3.readlines()


######### Mass1 ##########
inside_eventM1 = False
for lineM1 in linesM1:
    if "<event>" in lineM1:
        inside_eventM1 = True
        event_dataM1 = []
        continue
    elif "</event>" in lineM1:
        inside_eventM1 = False
        for pdata in event_dataM1:
            eventsM1.append(pdata)
        continue

    if inside_eventM1:
        try:
            split = lineM1.strip().split()
            if len(split) == 13:  # typical number of columns per particle
                pdg_id = int(split[0])
                status = int(split[1])
                px = float(split[6])
                py = float(split[7])
                pz = float(split[8])
                E = float(split[9])
                m = float(split[10])
                eventsM1.append({
                    "id": pdg_id,
                    "status": status,
                    "px": px,
                    "py": py,
                    "pz": pz,
                    "E": E,
                    "m": m
                })
        except:
            continue  # skip malformed lines

######### Mass2 ##########
inside_eventM2 = False
for lineM2 in linesM2:
    if "<event>" in lineM2:
        inside_eventM2 = True
        event_dataM2 = []
        continue
    elif "</event>" in lineM2:
        inside_eventM2 = False
        for pdata in event_dataM2:
            eventsM2.append(pdata)
        continue

    if inside_eventM2:
        try:
            split = lineM2.strip().split()
            if len(split) == 13:  # typical number of columns per particle
                pdg_id = int(split[0])
                status = int(split[1])
                px = float(split[6])
                py = float(split[7])
                pz = float(split[8])
                E = float(split[9])
                m = float(split[10])
                eventsM2.append({
                    "id": pdg_id,
                    "status": status,
                    "px": px,
                    "py": py,
                    "pz": pz,
                    "E": E,
                    "m": m
                })
        except:
            continue  # skip malformed lines


######### Mass3 ##########
inside_eventM3 = False
for lineM3 in linesM3:
    if "<event>" in lineM3:
        inside_eventM3 = True
        event_dataM3 = []
        continue
    elif "</event>" in lineM3:
        inside_eventM3 = False
        for pdata in event_dataM3:
            eventsM3.append(pdata)
        continue

    if inside_eventM3:
        try:
            split = lineM3.strip().split()
            if len(split) == 13:  # typical number of columns per particle
                pdg_id = int(split[0])
                status = int(split[1])
                px = float(split[6])
                py = float(split[7])
                pz = float(split[8])
                E = float(split[9])
                m = float(split[10])
                eventsM3.append({
                    "id": pdg_id,
                    "status": status,
                    "px": px,
                    "py": py,
                    "pz": pz,
                    "E": E,
                    "m": m
                })
        except:
            continue  # skip malformed lines

In [14]:
dfM1 = pd.DataFrame(eventsM1)
dfM2 = pd.DataFrame(eventsM2)
dfM3 = pd.DataFrame(eventsM3)

# filtering photon energy between 1GeV and 40GeV
signalM1 = dfM1[(dfM1['E'] >= 1) & (dfM1['E'] <= 40) & (dfM1['id']==22)][['E']]
signalM2 = dfM2[(dfM2['E'] >= 1) & (dfM2['E'] <= 40) & (dfM2['id']==22)][['E']]
signalM3 = dfM3[(dfM3['E'] >= 1) & (dfM3['E'] <= 40) & (dfM3['id']==22)][['E']]

# Turning Geant4 data into dataframe

In [15]:
df = pd.read_csv("gammaPiZeroExit.txt", delimiter=',', on_bad_lines="skip", index_col=False)

# Convert all columns to numeric, invalid parsing will be set as NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Drop rows where any element is NaN
df.dropna(inplace=True)
background = df[(df['energy'] >= 1000) & (df['energy'] <= 40000)]  # in MeV
background = background['energy'] / 1000


# Turning continuous data from background into histogram bins

In [16]:
num_bins = 12
bin_edges = np.linspace(1, 40, num_bins + 1)

# turning into arrays with 12 elements, each with the number of counts in each bin
background_hist , _ = np.histogram(background, bins=bin_edges)
signalM1_hist , _ = np.histogram(signalM1, bins=bin_edges)
signalM2_hist , _ = np.histogram(signalM2, bins=bin_edges)
signalM3_hist , _ = np.histogram(signalM3, bins=bin_edges)
signal_hist=[signalM1_hist, signalM2_hist, signalM3_hist]

background_hist = np.where(background_hist == 0, 0.1, background_hist) # avoid 0 so that everything scales smoothly
# Background in a year
H0 = background_hist * bckg_factor

## Chi2

# chi2

In [20]:
def sample_bins_in_chunks(size, p, n_bins, batch_size=100000):
    total_counts = np.zeros(n_bins, dtype=int)    # bin counts will be stored here
    full_batches = size // batch_size             # numer of batches of 100000 elements
    remainder = size % batch_size                 # the missing batches

    for _ in range(full_batches):
        bins = np.random.choice(n_bins, size=batch_size, p=p)    # samples bin indices 100000 times with probabilities p
        total_counts += np.bincount(bins, minlength=n_bins)      # adds the counts to each bin

    if remainder > 0:
        bins = np.random.choice(n_bins, size=remainder, p=p)    # does the same for the rest of the batch
        total_counts += np.bincount(bins, minlength=n_bins)

    return total_counts


for i in range(0,1):
  for e in eps2:
      # signal in a year for epsilon = e
      signal = signal_hist[i] * signal_factor(e, mA[i], signal_hist[i]) * eps_factor(e)

      # amount of pi0 that decay to gamma + A'
      factor = math.ceil(scale_factor(e, mA[i]) * np.sum(H0))

      # normalize the histograms to probability distributions
      H0_probs = H0 / np.sum(H0)
      signal_probs = signal / np.sum(signal)

      remove_counts = sample_bins_in_chunks(size=2*factor, p=H0_probs, n_bins=len(H0))
      add_counts = sample_bins_in_chunks(size=factor, p=signal_probs, n_bins=len(H0))

      H1 = H0 - remove_counts + add_counts
      H1 = np.clip(H1, 0, None)

      epsilon = 1e-12
      chi2_stat = np.sum((H0 - H1)**2 / H1+epsilon)

      print(f"Dark photon mass: ", mA[i])
      pvalue=chi2.sf(chi2_stat,12)
      print("Signal factor:", factor)
      print("Chi²:", chi2_stat)
      print(f"p-value: ", pvalue)
      print(f"eps2: ", e)
      print("")
      print("")

  print("............................")


Dark photon mass:  0.1
Signal factor: 55226463201
Chi²: 18.209871552796432
p-value:  0.10946510710618824
eps2:  1.3e-09


............................
